Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

<!--NOTEBOOK_HEADER-->
*This notebook contains material from [PyRosetta](https://RosettaCommons.github.io/PyRosetta.notebooks);
content is available [on Github](https://github.com/RosettaCommons/PyRosetta.notebooks.git).*

<!--NAVIGATION-->
< [`GALigandDock` Protocol with `pyrosetta.distributed` Using the `beta_cart.wts` Scorefunction](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/08.02-Ligand-Docking-pyrosetta.distributed.ipynb) | [Contents](toc.ipynb) | [Index](index.ipynb) | [Working With Density](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/11.00-Working-With-Density.ipynb) ><p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/10.00-Working-With-Symmetry.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>

# Working With Symmetry
Keywords: symmetry, asymmetric, SetupForSymmetryMover, virtual

## Overview
Symmetry is an important concept to learn when working with biomolecules.  When a protein is crystalized, it is in the precense of its symmetrical neighbors - which can be important if testing particular protocols or using crystal density for refinement or full structure building.

Symmetry can also be useful for designing symmetrical structures or large repeating meta-proteins like protein cages. 

### Symmetry In Rosetta
So why do we care if our protein is symmetrical or not when it comes to Rosetta?  Each residue and atom that is loaded into Rosetta takes time to both load, and time to score.  Since scoring can happen thousands of times - even in a short protocol, anything we can do to speed this up becomes important. The most expensive operation in Rosetta is minimization, and by using symmetry - we can reduce the minimization time exponentially by minimizing a single copy instead of ALL copies.  We will get into the details about how this works below.

When we use symmetry in Rosetta - we are basically telling rosetta that the symmetrical partners are 'special', however, the total number of residues is now ALL residues, including symmetrical partners.  Upon setting up symmety in Rosetta, Rosetta will replace the `Conformation` within the pose with a **Symmetrical** version, called the `SymmetricConformation`.  If you know anything about classes, this `SymmetricConformation` is derived from the actual `Conformation` object, but contains extra information about the pose and some functions are replaced. 

### Symmetric Scoring and Moving
Ok, so now lets assume that we have our symmetric pose.  Now what?  Well, the symmetric copies are all tied to their real counterparts.  Once you move a chain, residue, or atom by packing or minimization, the symmetric copies of that residue are all moved in the same way. 

Cool.  But what about scoring?  Scoring works very similarly - instead of scoring each and every residue in our pose, Rosetta will score just our assymetric unit, and multiply that out to the number of symmetric copies we have.  Intelligently, Rosetta will also figure out the symmetric interfaces that arise from the interactions of our assymetric unit to the symmetric copies and score them appropriately.

### Symmetry-aware movers
Most of our common movers are symmetry-aware.  At one point there were different symmetric and non-symmetric versions of particular code, such as MinMover and PackRotamersMover.  Now though, Rosetta will automatically use the pose to figure out what needs to be done. You should seek original documentation (and contact the author if not explicit) to make sure that an uncommon protocol you are using is symmetry-aware.

## Documentation
More information on RosettaSymmetry can be found in the following places:
- https://www.rosettacommons.org/docs/latest/rosetta_basics/structural_concepts/symmetry
- https://www.rosettacommons.org/demos/latest/tutorials/Symmetry/Symmetry
- https://www.rosettacommons.org/docs/latest/application_documentation/utilities/make-symmdef-file
- https://www.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/Movers/movers_pages/SetupForSymmetryMover
- https://www.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/Movers/movers_pages/ExtractAsymmetricUnitMover


In [ ]:
# Notebook setup
import sys
if 'google.colab' in sys.modules:
    !pip install pyrosettacolabsetup
    import pyrosettacolabsetup
    pyrosettacolabsetup.setup()
    print ("Notebook is set for PyRosetta use in Colab.  Have fun!")

Here, we will use a few specific options. The first three options make Rosetta a bit more robust to input structures.  The `-load_PDB_components` cannot be used with glycans, unfortunately, and our structure has a few very important glycans.  Finally, we load a bunch of glycan-specific options, which we will cover in the next tutorial.

In [ ]:
from pyrosetta import *
from pyrosetta.rosetta import *
from pyrosetta.teaching import *
import os

init('-ignore_unrecognized_res -load_PDB_components false -ignore_zero_occupancy false @inputs/glycan_flags')

## Creating a SymDef file

Here, we will start with how to create a basic symdef file for cyrstal symmetry.  Note that there are ways to do this without a symdef file, but these do not currently work for glycan structures, which we will be using here. 

The `make_symdef_file.pl` file is within Rosetta3. To use it, you will need to download and licence Rosetta3.  The code is in the `Rosetta/main/src/apps/public` directory. In the interest of reducing code drift, this file is NOT included in the tutorial directory as we may then have version drift.  

If you have done this, we can use the following command to create the symdef file.  Here, the radius of symmetrical partners is 12A, which is certainly fairly large, but produces a very well represented crystal.

In [ ]:
pdb = "inputs/1jnd.pdb"
base_cmd = f'cd inputs && make_symmdef_file.pl -r 12 -m CRYST -p  {pdb}.pdb > {pdb}_crys.symm && cd -'
print(base_cmd)

Use this base command and the `os.system(cmd)` function to run the code or use the provided symdef file. 

In [ ]:
os.system('cp inputs/1jnd_crys.symm .')

Take a look at the symmetrized structure in pymol (`inputs/1jnd_symm.pdb`).  What would happen if we increased the radius to 24 instead of 12?

## Setup a Symmetrized Pose

Here, we will run a basic Rosetta protocol with symmetry.  There are much more complicated things you can do with symmetry, but for now, we just want to symmetrically pack the protein.  Please see the docs for more on symmetry.  The full Rosetta C++ tutorial for symmetry is a great place to go from here: - https://www.rosettacommons.org/demos/latest/tutorials/Symmetry/Symmetry 

Lets first create a pose, and then use the `SetupForSymmetryMover` on the pose. Note this is an unrefined input structure.  This is so that minmover will actually do something.  A pareto-optimal refined structure can be found in the inputs as `1jnd_refined.pdb.gz`

In [ ]:
p = pose_from_pdb('inputs/1jnd.pdb')
original = p.clone()

In [ ]:
p.total_residue()

In [ ]:
type(p.conformation())

In [ ]:
symmetrize = rosetta.protocols.symmetry.SetupForSymmetryMover("1jnd_crys.symm")
symmetrize.apply(p)

In [ ]:
print(p.total_residue())
print(type(p.conformation()))

How many symmetric copies do we have in our pose?
How do the scores compare for our original pose and our symmetrized version?

Now lets use some of the functionality to understand how this all works. We can use the `SymetricInfo` object that is part of the `SymmetricConformation` to get at some info. Lets take a look at all residues and find the assymetric unit residues and equivalent residues for the rest.

In [ ]:
print("AssymUnit? equivalent_res")
sym_info = p.conformation().Symmetry_Info()
for i in range(1, p.size()+1):
    print(i, sym_info.bb_is_independent(i), sym_info.bb_follows(i))

Which residues are our original pose residues?  Note that the final residues are called `Virtual` residues.  Virtual residues are not scored.  They have coordinates, and can move, but simply result in a score of zero.  They are useful in some contexts to hide a part of the pose from the scoring machinery, and there are movers that can change residues to and from virtual.  In this case, they are used for the FoldTree - in order to allow refinement of the full crystal environment.  They allow relative movement of each subunit relative to each other.  There are two virtual residues for each subunit

In [ ]:
print(p.residue(3654))

In [ ]:
print("Total Subunits:", (3654-18)/404)
print("Total Subunits:", sym_info.subunits())

In [ ]:
score = get_score_function()
print(score(original))
print(score(p))

## Running Protocols with Symmetry

Now, lets try running a minimization with symmetry on.  

In [ ]:
mm = MoveMap()
mm.set_bb(True)
mm.set_chi(True)
minmover = rosetta.protocols.minimization_packing.MinMover()
minmover.score_function(score)
minmover.set_movemap(mm)
minmover.apply(p)

In [ ]:
score(p)

How does our pose look?  For being such a large pose, how was the speed of minimization? 

How does this compare to our refined pose?  Try to copy a subunit to a new object in PyMol.  Then use the align command to align it to our assymetric unit.  What is the RMSD?

Now lets pack with our symmetric structure. 

In [ ]:
from rosetta.core.pack.task import *
from rosetta.core.pack.task.operation import *

packer = PackRotamersMover()
tf = rosetta.core.pack.task.TaskFactory()

tf.push_back(RestrictToRepacking())
tf.push_back(IncludeCurrent())
packer.task_factory(tf)

p = original.clone()
symmetrize.apply(p)

In [ ]:
packer.apply(p)

In [ ]:
print("packed", score(p))

## Conclusions

Symmetry is a useful tool in the Rosetta Library.  There are also selectors and movers that you may find useful, such as the `AsymmetricUnitSelector` in `rosetta.core.select.residue_selectors` and the `ExtractAsymmetricUnitMover`, which will give you back just the single subunit, without any asymetric partners, and the `ExtractAsymmetricPoseMover`, which will remove 'symmetry' information and give you back a pose with all the subunits. The later of these can be found by importing `rosetta.protocols.symmetry`.

Note that not ALL protocols will respect symmetry - so please check the original documentation to see if symmetry is supported.  If you are unsure, please email the developer.

**Chapter contributors:**

- Jared Adolf-Bryfogle (Scripps; Institute for Protein Innovation)

<!--NAVIGATION-->
< [`GALigandDock` Protocol with `pyrosetta.distributed` Using the `beta_cart.wts` Scorefunction](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/08.02-Ligand-Docking-pyrosetta.distributed.ipynb) | [Contents](toc.ipynb) | [Index](index.ipynb) | [Working With Density](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/11.00-Working-With-Density.ipynb) ><p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/10.00-Working-With-Symmetry.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>